# ATENCION

In [2]:
import torch
import random
import numpy as np
import torch.nn.functional as F

Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

S =torch.mm(Q,K.T)
S=S/np.sqrt(K.shape[1])

n = S.shape[0]
print(S)

S = F.softmax(S, dim = 1)
print(S)
A = torch.sum(S[:, :, None] * V[None, :, :], dim=1)

print('\n',A)

torch.Size([4, 3])
torch.Size([4, 3])
torch.Size([4, 3])
torch.Size([4, 4])
tensor([[0.0000, 0.0000, 0.0000, 0.0000],
        [0.1732, 0.3464, 0.5196, 0.6928],
        [0.0346, 0.0693, 0.1039, 0.1386],
        [0.0520, 0.1039, 0.1559, 0.2078]])
tensor([[0.2500, 0.2500, 0.2500, 0.2500],
        [0.1892, 0.2250, 0.2676, 0.3182],
        [0.2372, 0.2455, 0.2542, 0.2631],
        [0.2309, 0.2432, 0.2561, 0.2698]])

 tensor([[0.2500, 0.5000, 0.5000],
        [0.1892, 0.5432, 0.5857],
        [0.2372, 0.5087, 0.5173],
        [0.2309, 0.5130, 0.5260]])


# ATENCION CON ENMASCARAMENTO

In [38]:
import torch
import random
import numpy as np
import torch.nn.functional as F


Q = torch.tensor([[0.0, 0.0, 0.0], [1, 1, 1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3]])
K = torch.tensor([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4]])
V = torch.tensor([[1., 0., 0.], [0., 1., 0.], [0., 0., 1.], [0., 1., 1.]])

S =torch.mm(Q,K.T)
S=S/np.sqrt(K.shape[1])

n = S.shape[0]
mask = torch.tril(torch.ones(n,n))==1  #triangular matrix
S = torch.where(mask, S, float('-inf'))
print(S)

S = F.softmax(S, dim = 1)
print(S)
A = torch.sum(S[:, :, None] * V[None, :, :], dim=1)

print('\n',A)

tensor([[0.0000,   -inf,   -inf,   -inf],
        [0.1732, 0.3464,   -inf,   -inf],
        [0.0346, 0.0693, 0.1039,   -inf],
        [0.0520, 0.1039, 0.1559, 0.2078]])
tensor([[1.0000, 0.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000, 0.0000],
        [0.3219, 0.3332, 0.3449, 0.0000],
        [0.2309, 0.2432, 0.2561, 0.2698]])

 tensor([[1.0000, 0.0000, 0.0000],
        [0.4568, 0.5432, 0.0000],
        [0.3219, 0.3332, 0.3449],
        [0.2309, 0.5130, 0.5260]])


# EINSTEIN SUM

In [19]:
from torch import nn

batch_size = 3
sequence_size = 4
embedding_size = 5
hidden_size = 5

x = torch.randn(batch_size, sequence_size,embedding_size)
encoder_states = torch.randn(batch_size, sequence_size,embedding_size)
hidden = torch.randn(1, sequence_size,embedding_size)
cell = torch.randn(1, sequence_size,embedding_size)

def attention(x, encoder_states, hidden, cell):
    #sequence_length = encoder_states.shape[0]

    Q = torch.randn(hidden_size, hidden_size)
    K = torch.randn(hidden_size, hidden_size)
    V = torch.randn(hidden_size, embedding_size)

    Q = torch.matmul(hidden, Q)
    K = torch.matmul(encoder_states, K)
    V = torch.matmul(encoder_states, V)
        
    S = torch.matmul(Q, K.transpose(-2, -1))  # Transpose K along the last two dimensions
    S = S / np.sqrt(K.shape[-1])

    S = F.softmax(S, dim=-1)
    context_vector = torch.sum(S.unsqueeze(-1) * V.unsqueeze(1), dim=-2)
    return context_vector.shape, context_vector


def EinsteinSum(x, encoder_states, hidden, cell):
    print(x.shape)
    print(encoder_states.T.shape)
    print(hidden.T.shape)
    print(cell.shape)

    energyfun = nn.Linear(hidden_size * 3, 1)
    energy = nn.ReLU(energyfun(torch.cat((hidden, encoder_states), dim=2)))

    attention = nn.Softmax(energy)
    # attention: (seq_length, N, 1)

    # attention: (seq_length, N, 1), snk
    # encoder_states: (seq_length, N, hidden_size*2), snl
    # we want context_vector: (1, N, hidden_size*2), i.e knl
    context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)

print(attention(x, encoder_states, hidden, cell))
print(EinsteinSum(x, encoder_states, hidden, cell))

(torch.Size([3, 4, 5]), tensor([[[-1.7706e-02,  2.8828e+00, -7.1356e+00,  4.5344e+00, -1.5819e-01],
         [-4.4755e-02,  1.4575e+00, -4.1217e-01, -1.8539e+00, -2.6193e+00],
         [-1.7705e-02,  2.8828e+00, -7.1356e+00,  4.5344e+00, -1.5820e-01],
         [ 4.7722e-01,  1.0455e+00,  1.3967e+00, -4.7866e-01, -2.3153e+00]],

        [[-3.3266e-03,  5.6681e+00, -3.7679e+00,  2.1016e+00, -2.5847e+00],
         [ 4.2982e-01,  2.9642e+00, -1.3427e+00,  9.1846e-01, -2.3861e-01],
         [ 1.7429e+00,  2.8946e+00,  8.1312e-01,  3.0068e+00, -2.4894e-01],
         [ 1.1529e+00,  2.9465e+00, -2.5406e-01,  2.0860e+00, -4.9052e-01]],

        [[-5.6236e-01,  1.4978e-01,  2.3013e+00,  1.3614e+00,  4.1825e+00],
         [ 2.5039e-01, -1.1190e+00,  9.3869e-01, -1.5250e+00, -2.4084e+00],
         [ 1.8348e+00,  3.9938e-01, -3.3142e-01,  3.7015e+00, -4.4636e+00],
         [ 1.2122e+00, -1.6552e-01,  6.8812e-01,  1.3289e+00, -3.5608e+00]]]))
torch.Size([3, 4, 5])
torch.Size([5, 4, 3])
torch.Size([5

RuntimeError: mat1 and mat2 shapes cannot be multiplied (20x4 and 15x1)

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

def EinsteinSum(x, encoder_states, hidden, cell):
    print(x.shape)
    print(encoder_states.T.shape)
    print(hidden.T.shape)
    print(cell.shape)

    energyfun = nn.Linear(hidden_size * 3, 1)
    energy = F.relu(energyfun(torch.cat((hidden, encoder_states), dim=2)))

    # Apply Softmax along the last dimension (dim=2)
    attention = F.softmax(energy, dim=2)
    # attention: (seq_length, N, 1)

    # attention: (seq_length, N, 1), snk
    # encoder_states: (seq_length, N, hidden_size*2), snl
    # we want context_vector: (1, N, hidden_size*2), i.e knl
    context_vector = torch.einsum("snk,snl->knl", attention, encoder_states)
    return context_vector

# Assuming hidden_size is defined somewhere in your code
hidden_size = 256

# Assuming you have defined x, encoder_states, hidden, cell
# For illustration purposes, you can replace these with actual tensors in your implementation.
x = torch.randn(10, 5, hidden_size)  # seq_length=10, batch_size=5, hidden_size=256
encoder_states = torch.randn(10, 5, hidden_size * 2)  # seq_length=10, batch_size=5, hidden_size*2=512
hidden = torch.randn(5, 1, hidden_size * 3)  # N=5, hidden_size*3=768
cell = torch.randn(5, 1, hidden_size)  # N=5, hidden_size=256

print(EinsteinSum(x, encoder_states, hidden, cell))


torch.Size([10, 5, 256])
torch.Size([512, 5, 10])
torch.Size([768, 1, 5])
torch.Size([5, 1, 256])


RuntimeError: Sizes of tensors must match except in dimension 2. Expected size 5 but got size 10 for tensor number 1 in the list.